Data generation for 1D burgers equation using finite differences discretization in space, and an ODE solver for time.

In [1]:
using OrdinaryDiffEq
using ComponentArrays
using Plots
using JLD2

In [2]:
struct burgers_equation
    Δx::Float64 # grid spacing
    ν::Float64 # viscosity
    du_dx::Vector{Float64} # temporary storage for du/dx
    d2u_dx2::Vector{Float64} # temporary storage for d2u/dx2
end

function dx!(du,u,Δx) #upwind du_dx for u*du/dx term
    n=length(u)
    for i=2:n
        du[i]=(u[i]-u[i-1])/Δx
    end
    du[1]=(u[1]-u[end])/Δx
end

function dx2!(du,u,Δx) #central second difference for ν d2u/dx2 term
    n=length(u)
    for i=2:n-1
        du[i]=(u[i+1]-2*u[i]+u[i-1])/(Δx^2)
    end
    du[1]=(u[end]-2*u[1]+u[2])/(Δx^2)
    du[end]=(u[end-1]-2*u[end]+u[1])/(Δx^2)
end

"""
# Define the function that computes the time derivative of the state
 du/dt = -u * du/dx + ν * d2u/dx2
"""
function (f::burgers_equation)(du_dt, u, p, t)
    Δx=f.Δx
    ν=f.ν
    du_dx=f.du_dx
    d2u_dx2=f.d2u_dx2
    dx!(du_dx,u,Δx)
    dx2!(d2u_dx2,u,Δx)
    @. du_dt = ν*d2u_dx2 - u*du_dx
end

function burgers_initial_condition(x,k,u_mean=1.0,u_amplitude=0.5)
    return u_mean .+ u_amplitude.*cos.(k*x)
end;

In [3]:
# setting up the problem

# parameters
nx=100     # number of spatial points
L=10.0     # length of the domain
Δx=L/nx    # spatial step
ν=0.1    # viscosity
k=2*pi/L   # wave number for initial condition
u_mean=1 # mean velocity
u_amplitude=0.5 # amplitude of the initial condition
p=()       # parameters (not used in this case)

x=Δx*(1:nx) # spatial grid points

# initial condition
u0 = burgers_initial_condition(x,k,u_mean,u_amplitude)

# initialize model struct
f=burgers_equation(Δx, ν, similar(u0), similar(u0))

# time span
t_end=15;

In [4]:
prob = ODEProblem(f, u0, (0.0, t_end))
sol = solve(prob, Tsit5());

In [9]:
# save the solution to a JLD2 file
cfl_out=0.2283 # CFL number for output
nx_output=100 # at how many points to save, should be a multiple of nx for clean result
Δx_output=L/nx_output
dt_out=cfl_out*Δx_output/u_mean # time step for output, based on CFL condition
println(dt_out)
# spatial grid
x_output=Δx_output*(1:nx_output) # spatial grid points

times=collect(0.0:dt_out:t_end) # output times
solution=[]
for t in times
    push!(solution, sol(t)[1:nx÷nx_output:end])
end


# save the solution to a JLD2 file
save("data/burgers1d_periodicTEMP2.jld2", 
    "solution", solution,
    "times", times,
    "params", (ν=ν, Δx=Δx, u_mean=u_mean, u_amplitude=u_amplitude, k=k),
    "grid", x_output,
    "cfl_out", cfl_out,
    "dt_out", dt_out,
    "description", "Solution of the 1D Burgers equation with periodic boundary conditions, using a cosine as initial condition.")


0.022830000000000003
